In [2]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:0000:01:00:01
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/4d/d3/38b09813a32618acd437906c4d0194119e27139dbcd7486e69d58e375a27/regex-2023.10.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.6 MB/s eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━

In [25]:
import nltk
nltk.download('reuters')

from nltk.corpus import reuters

# Define the ten specified classes
specified_classes = [
    'acquisitions', 'corn', 'crude', 'earn', 'grain',
    'interest', 'money-fx', 'ship', 'trade', 'wheat'
]

# Get documents that belong to only one of the specified classes
single_class_documents = []

for doc_id in reuters.fileids():
    categories = reuters.categories(doc_id)
    relevant_categories = [category for category in categories if category in specified_classes]

    # If the document belongs to exactly one of the specified classes
    if len(relevant_categories) == 1:
        single_class_documents.append(doc_id)

# Display the number of filtered documents
print(f"Number of documents belongs to one of these ten classes: {len(single_class_documents)}")


Number of documents belongs to one of these ten classes: 5928


[nltk_data] Downloading package reuters to
[nltk_data]     /Users/amitkumar/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [10]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 10.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [27]:
document_texts = [reuters.raw(doc_id) for doc_id in single_class_documents]

In [28]:
vectorizer = TfidfVectorizer(stop_words='english')

In [29]:
tfidf_matrix = vectorizer.fit_transform(document_texts)

In [30]:
num_clusters = 10  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=10, random_state=42)

In [31]:
cluster_labels = kmeans.labels_

# Print cluster assignments for the first few documents
for i in range(50):
    print(f"Document ID: {single_class_documents[i]} - Cluster: {cluster_labels[i]}")

Document ID: test/14826 - Cluster: 3
Document ID: test/14828 - Cluster: 2
Document ID: test/14829 - Cluster: 3
Document ID: test/14839 - Cluster: 2
Document ID: test/14859 - Cluster: 2
Document ID: test/14860 - Cluster: 0
Document ID: test/14862 - Cluster: 3
Document ID: test/14872 - Cluster: 2
Document ID: test/14873 - Cluster: 2
Document ID: test/14875 - Cluster: 2
Document ID: test/14876 - Cluster: 2
Document ID: test/14881 - Cluster: 3
Document ID: test/14899 - Cluster: 2
Document ID: test/14903 - Cluster: 2
Document ID: test/14904 - Cluster: 3
Document ID: test/14911 - Cluster: 2
Document ID: test/14912 - Cluster: 3
Document ID: test/14913 - Cluster: 3
Document ID: test/14926 - Cluster: 2
Document ID: test/14930 - Cluster: 2
Document ID: test/14931 - Cluster: 3
Document ID: test/14933 - Cluster: 2
Document ID: test/14934 - Cluster: 9
Document ID: test/14954 - Cluster: 0
Document ID: test/14957 - Cluster: 2
Document ID: test/14958 - Cluster: 0
Document ID: test/14959 - Cluster: 2
D

In [18]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [32]:
# Encode true class labels
true_labels = [reuters.categories(doc_id)[0] for doc_id in single_class_documents]
label_encoder = LabelEncoder()
true_labels_encoded = label_encoder.fit_transform(true_labels)

# Compute purity
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

purity = purity_score(true_labels_encoded, cluster_labels)

# Compute normalized mutual information (NMI)
nmi = metrics.normalized_mutual_info_score(true_labels_encoded, cluster_labels)

# Compute F1 score
f1_score = metrics.f1_score(true_labels_encoded, cluster_labels, average='weighted')

# Compute Rand Index (RI)
ri = metrics.adjusted_rand_score(true_labels_encoded, cluster_labels)

# Print the computed metrics
print(f"Purity: {purity:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Rand Index (RI): {ri:.4f}")

Purity: 0.7799
Normalized Mutual Information (NMI): 0.4084
F1 Score: 0.0003
Rand Index (RI): 0.1015


In [21]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 11.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [33]:
# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels_encoded, cluster_labels)

# Ensure that the confusion matrix only includes the ten specified classes and clusters
conf_matrix = conf_matrix[:len(specified_classes), :num_clusters]

# Create a DataFrame for better visualization
conf_matrix_df = pd.DataFrame(conf_matrix, index=specified_classes, columns=[f'Cluster_{i}' for i in range(num_clusters)])

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_df)


Confusion Matrix:
              Cluster_0  Cluster_1  Cluster_2  Cluster_3  Cluster_4  \
acquisitions          0          0         29          3         14   
corn                  0          0          4          0          0   
crude                 0          0         16          0          0   
earn                  1          0          4         16          1   
grain                 0          0          1          8          0   
interest              0          0          1          0          0   
money-fx              0          0          0          2          0   
ship                  0          0          0          1          0   
trade                 0          0          5          2          1   
wheat                 0          0          2          0          0   

              Cluster_5  Cluster_6  Cluster_7  Cluster_8  Cluster_9  
acquisitions          0          0          0          0          0  
corn                  0          0          0          0    

In [34]:
# Iterate through the confusion matrix to identify FPs
for i in range(len(specified_classes)):
    # False Positives (FPs)
    fp_count = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
    print(f"False Positives for class '{specified_classes[i]}': {fp_count}")
     # False Negatives (FNs)
    fn_count = np.sum(conf_matrix[i, :]) - conf_matrix[i, i]
    print(f"False Negatives for class '{specified_classes[i]}': {fn_count}")

False Positives for class 'acquisitions': 1
False Negatives for class 'acquisitions': 46
False Positives for class 'corn': 0
False Negatives for class 'corn': 4
False Positives for class 'crude': 46
False Negatives for class 'crude': 0
False Positives for class 'earn': 16
False Negatives for class 'earn': 39
False Positives for class 'grain': 16
False Negatives for class 'grain': 9
False Positives for class 'interest': 0
False Negatives for class 'interest': 1
False Positives for class 'money-fx': 0
False Negatives for class 'money-fx': 2
False Positives for class 'ship': 0
False Negatives for class 'ship': 1
False Positives for class 'trade': 33
False Negatives for class 'trade': 8
False Positives for class 'wheat': 0
False Negatives for class 'wheat': 2
